## label as normalized order of finish based on number of horses in race
the label should be resricted to a 0 - 1 range taking into account the number of horses per race
so a horse that places 5th in a race of 10 horses gets a score of 0.5

$
\text{label} = 1 - \dfrac{(\text{finish order} - 1)}{(\text{number of horses in race} - 1)}
$

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import display

sys.path.insert(0, '..')
np.random.seed(42)
pd.options.display.max_columns = None
%matplotlib inline

In [2]:
from scripts.utils import read_netkeiba

df = read_netkeiba()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240471 entries, 0 to 240470
Data columns (total 66 columns):
c_id                                      240471 non-null int64
c_weight_carried                          240471 non-null float64
c_post_position                           240471 non-null int64
c_order_of_finish                         240471 non-null int64
c_order_of_finish_lowered                 240471 non-null int64
c_finish_time                             240471 non-null float64
c_horse_weight                            240471 non-null int64
c_horse_weight_diff                       240471 non-null int64
c_popularity                              240471 non-null int64
c_first_place_odds                        240471 non-null float64
r_id                                      240471 non-null int64
r_key                                     240471 non-null object
r_racetrack                               240471 non-null object
r_course_type                             240471 

In [4]:
df.head()

,c_id,c_weight_carried,c_post_position,c_order_of_finish,c_order_of_finish_lowered,c_finish_time,c_horse_weight,c_horse_weight_diff,c_popularity,c_first_place_odds,r_id,r_key,r_racetrack,r_course_type,r_weather,r_url,r_distance,r_date,r_dirt_condition,r_turf_condition,r_impost_category,r_is_non_winner_regional_horse_allowed,r_is_winner_regional_horse_allowed,r_is_regional_jockey_allowed,r_is_foreign_horse_allowed,r_is_foreign_horse_and_trainer_allowed,r_is_apprentice_jockey_allowed,r_is_female_only,h_id,h_key,h_url,h_total_races,h_total_wins,h_sex,h_birthday,h_user_rating,j_id,j_key,j_url,j_career_1st_place_count,j_career_2nd_place_count,j_career_3rd_place_count,j_career_4th_place_or_below_count,j_career_turf_race_count,j_career_turf_win_count,j_career_dirt_race_count,j_career_dirt_win_count,j_career_1st_place_rate,j_career_1st_2nd_place_rate,j_career_any_place_rate,j_career_earnings,t_id,t_key,t_url,t_career_1st_place_count,t_career_2nd_place_count,t_career_3rd_place_count,t_career_4th_place_or_below_count,t_career_turf_race_count,t_career_turf_win_count,t_career_dirt_race_count,t_career_dirt_win_count,t_career_1st_place_rate,t_career_1st_2nd_place_rate,t_career_any_place_rate,t_career_earnings
0,1,54.0,7,1,0,109.9,514,-2,1,3.7,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,1,2016104408,http://db.netkeiba.com/horse/2016104408/,2,1,female,2016-04-03,3.17,1,01092,http://db.netkeiba.com/jockey/result/01092,423,477,565,6501,4156,222,3810,201,0.053,0.113,0.184,829574.0,1,00359,http://db.netkeiba.com/trainer/result/00359,556,577,557,5185,3613,299,3052,241,0.081,0.165,0.246,892578.6
1,2,54.0,8,2,0,110.1,490,0,3,6.3,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,2,2016100908,http://db.netkeiba.com/horse/2016100908/,3,0,female,2016-02-07,3.25,2,01109,http://db.netkeiba.com/jockey/result/01109,142,182,189,3746,2091,74,2167,68,0.033,0.076,0.120,253625.0,2,01147,http://db.netkeiba.com/trainer/result/01147,92,84,81,866,508,36,537,45,0.082,0.157,0.229,136755.2
2,3,54.0,4,3,0,110.4,464,10,4,6.4,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,3,2016105892,http://db.netkeiba.com/horse/2016105892/,2,0,female,2016-04-29,NaN,3,01128,http://db.netkeiba.com/jockey/result/01128,330,299,350,4005,2498,146,2486,184,0.066,0.126,0.196,521757.2,3,01006,http://db.netkeiba.com/trainer/result/01006,286,255,279,3530,1854,111,2237,154,0.066,0.124,0.189,454950.2
3,4,54.0,6,4,0,110.4,450,-6,2,4.5,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,4,2016106313,http://db.netkeiba.com/horse/2016106313/,3,0,female,2016-03-19,NaN,4,01015,http://db.netkeiba.com/jockey/result/01015,461,528,542,7051,4271,197,4311,264,0.054,0.115,0.178,852023.5,4,01099,http://db.netkeiba.com/trainer/result/01099,269,246,289,2686,1831,136,1558,118,0.077,0.148,0.230,493068.1
4,5,54.0,6,5,0,110.5,426,-8,10,43.3,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,5,2016102321,http://db.netkeiba.com/horse/2016102321/,2,0,female,2016-05-21,NaN,5,01117,http://db.netkeiba.com/jockey/result/01117,372,429,435,5399,3408,201,3227,171,0.056,0.121,0.186,660899.9,5,01131,http://db.netkeiba.com/trainer/result/01131,60,71,77,1541,703,16,1008,43,0.034,0.075,0.119,94999.5


In [5]:
df['r_contender_count'] = df.groupby('r_id').c_id.count().loc[df.r_id].values
df['c_norm_order_of_finish'] = 1.0 - (df.c_order_of_finish - 1) / (df.r_contender_count - 1)

index_cols = ['c_id', 'r_id', 'h_id', 'j_id', 't_id', 'r_key', 'r_url', 'h_key', 'h_url', 'j_key', 'j_url', 't_key', 't_url']
label_cols = ['c_norm_order_of_finish', 'c_order_of_finish', 'c_finish_time']

X = df.drop(columns=index_cols + label_cols)
y = df.c_norm_order_of_finish

X.shape, y.shape

((240471, 52), (240471,))

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((192376, 52), (192376,), (48095, 52), (48095,))

In [81]:
# fillna('')
# OneHotEncoder(handle_unknown='ignore')
cat_attribs = [
    'r_racetrack', 'r_course_type', 'r_weather', 'h_sex', 
    'r_impost_category', 'r_dirt_condition', 'r_turf_condition'
]

bool_attribs = [
    'r_is_non_winner_regional_horse_allowed', 'r_is_winner_regional_horse_allowed',
    'r_is_regional_jockey_allowed', 'r_is_foreign_horse_allowed',
    'r_is_foreign_horse_and_trainer_allowed', 'r_is_apprentice_jockey_allowed',
    'r_is_female_only', 'c_order_of_finish_lowered'
]

# CombinedDateAttributesAdder
date_attribs = [
    'r_date', 'h_birthday'
]

# Imputer(strategy='median')
# CombinedNumericAttributesAdder
# StandardScaler
num_attribs = [
    'c_weight_carried', 'c_post_position',
    'c_horse_weight', 'c_horse_weight_diff', 'c_popularity',
    'c_first_place_odds', 'r_distance', 'r_contender_count',
    'h_total_races', 'h_total_wins', 'h_user_rating',
    'j_career_1st_place_count', 'j_career_2nd_place_count',
    'j_career_3rd_place_count', 'j_career_4th_place_or_below_count',
    'j_career_turf_race_count', 'j_career_turf_win_count',
    'j_career_dirt_race_count', 'j_career_dirt_win_count',
    'j_career_1st_place_rate', 'j_career_1st_2nd_place_rate',
    'j_career_any_place_rate', 'j_career_earnings',
    't_career_1st_place_count', 't_career_2nd_place_count',
    't_career_3rd_place_count', 't_career_4th_place_or_below_count',
    't_career_turf_race_count', 't_career_turf_win_count',
    't_career_dirt_race_count', 't_career_dirt_win_count',
    't_career_1st_place_rate', 't_career_1st_2nd_place_rate',
    't_career_any_place_rate', 't_career_earnings'
]

In [88]:
from sklearn.base import BaseEstimator, TransformerMixin


class CombinedDateAttributesAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = pd.DataFrame(X)
        df['h_age_days'] = pd.to_datetime(df['r_date']) - pd.to_datetime(df['h_birthday'])
        return df.drop(columns=['r_date', 'h_birthday']).values

    
class CombinedNumericAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.j_turf_wins_ix = columns.index('j_career_turf_win_count')
        self.j_turf_races_ix = columns.index('j_career_turf_race_count')
        self.j_dirt_wins_ix = columns.index('j_career_dirt_win_count')
        self.j_dirt_races_ix = columns.index('j_career_dirt_race_count')
        self.t_turf_wins_ix = columns.index('t_career_turf_win_count')
        self.t_turf_races_ix = columns.index('t_career_turf_race_count')
        self.t_dirt_wins_ix = columns.index('t_career_dirt_win_count')
        self.t_dirt_races_ix = columns.index('t_career_dirt_race_count')
        self.h_total_wins_ix = columns.index('h_total_wins')
        self.h_total_races_ix = columns.index('h_total_races')
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        j_turf_wins, j_turf_races = X[:, self.j_turf_wins_ix], X[:, self.j_turf_races_ix]
        j_turf_win_rate = np.divide(j_turf_wins, j_turf_races, out=np.zeros_like(j_turf_wins), where=j_turf_wins!=0.)

        j_dirt_wins, j_dirt_races = X[:, self.j_dirt_wins_ix], X[:, self.j_dirt_races_ix]
        j_dirt_win_rate = np.divide(j_dirt_wins, j_dirt_races, out=np.zeros_like(j_dirt_wins), where=j_dirt_wins!=0.)

        t_turf_wins, t_turf_races = X[:, self.t_turf_wins_ix], X[:, self.t_turf_races_ix]
        t_turf_win_rate = np.divide(t_turf_wins, t_turf_races, out=np.zeros_like(t_turf_wins), where=t_turf_wins!=0.)

        t_dirt_wins, t_dirt_races = X[:, self.t_dirt_wins_ix], X[:, self.t_dirt_races_ix]
        t_dirt_win_rate = np.divide(t_dirt_wins, t_dirt_races, out=np.zeros_like(t_dirt_wins), where=t_dirt_wins!=0.)

        h_wins, h_races = X[:, self.h_total_wins_ix], X[:, self.h_total_races_ix]
        h_win_rate = np.divide(h_wins, h_races, out=np.zeros_like(h_wins), where=h_wins!=0.)

        return np.c_[X, j_turf_win_rate, j_dirt_win_rate, t_turf_win_rate, t_dirt_win_rate, h_win_rate]

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('num_attribs_adder', CombinedNumericAttributesAdder(num_attribs)),
    ('std_scaler', StandardScaler())
])

added_columns = ['j_career_turf_win_rate', 'j_career_dirt_win_rate', 
                 't_career_turf_win_rate', 't_career_dirt_win_rate', 'h_win_rate']

X_train_num = num_pipeline.fit_transform(X[num_attribs])
X_train_num_df = pd.DataFrame(X_train_num, columns=num_attribs+added_columns)
X_train_num_df.describe()

,c_weight_carried,c_post_position,c_horse_weight,c_horse_weight_diff,c_popularity,c_first_place_odds,r_distance,r_contender_count,h_total_races,h_total_wins,h_user_rating,j_career_1st_place_count,j_career_2nd_place_count,j_career_3rd_place_count,j_career_4th_place_or_below_count,j_career_turf_race_count,j_career_turf_win_count,j_career_dirt_race_count,j_career_dirt_win_count,j_career_1st_place_rate,j_career_1st_2nd_place_rate,j_career_any_place_rate,j_career_earnings,t_career_1st_place_count,t_career_2nd_place_count,t_career_3rd_place_count,t_career_4th_place_or_below_count,t_career_turf_race_count,t_career_turf_win_count,t_career_dirt_race_count,t_career_dirt_win_count,t_career_1st_place_rate,t_career_1st_2nd_place_rate,t_career_any_place_rate,t_career_earnings,j_career_turf_win_rate,j_career_dirt_win_rate,t_career_turf_win_rate,t_career_dirt_win_rate,h_win_rate
count,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05,2.404710e+05
mean,1.794034e-15,-1.315475e-16,-7.453177e-16,-2.529305e-17,-9.266240e-17,-8.923484e-17,-1.954302e-16,-1.891069e-17,-4.538567e-17,6.429636e-17,-2.692410e-16,-2.600220e-17,-6.736935e-17,8.734377e-17,6.807850e-17,2.174730e-17,4.254906e-17,3.640309e-17,-7.942492e-17,4.188719e-16,2.651043e-16,1.705508e-16,-9.289878e-17,-5.578655e-17,9.171687e-17,1.132278e-16,-3.025711e-17,-4.302183e-17,1.181918e-17,2.411113e-17,6.666020e-17,-8.338434e-16,-9.651545e-16,-4.120167e-16,4.758403e-16,1.310747e-16,2.919338e-16,-6.760573e-16,4.896097e-16,2.486756e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-4.496857e+00,-1.644975e+00,-4.456702e+00,-7.555114e+00,-1.544014e+00,-6.893252e-01,-3.924796e+00,-4.171941e+00,-1.446032e+00,-1.101964e+00,-3.293532e+00,-9.329730e-01,-1.050803e+00,-1.145341e+00,-1.500885e+00,-1.334453e+00,-8.828953e-01,-1.441830e+00,-9.641900e-01,-1.811911e+00,-2.141232e+00,-2.476202e+00,-8.899300e-01,-1.464802e+00,-1.635806e+00,-1.752109e+00,-2.104924e+00,-1.736076e+00,-1.155583e+00,-2.128275e+00,-1.671901e+00,-2.875665e+00,-3.482781e+00,-4.053646e+00,-1.255234e+00,-1.602840e+00,-1.903030e+00,-2.507630e+00,-2.932909e+00,-1.149225e+00
25%,-5.628975e-01,-7.652313e-01,-6.781131e-01,-6.694141e-01,-8.717282e-01,-6.084657e-01,-5.876076e-01,-7.417900e-01,-7.395030e-01,-6.839368e-01,2.596911e-01,-7.486741e-01,-8.197801e-01,-8.596617e-01,-9.454290e-01,-9.484583e-01,-7.516211e-01,-8.913100e-01,-7.645817e-01,-7.011780e-01,-7.168785e-01,-7.401268e-01,-7.358911e-01,-7.184793e-01,-7.777596e-01,-8.010489e-01,-7.305498e-01,-7.480102e-01,-6.858802e-01,-7.617424e-01,-7.905280e-01,-6.320579e-01,-6.508884e-01,-6.415498e-01,-7.066177e-01,-6.648138e-01,-7.583222e-01,-7.410970e-01,-6.267767e-01,-6.543154e-01
50%,-9.032722e-04,1.145121e-01,-1.520274e-02,-4.344137e-02,2.465251e-02,-4.299235e-01,-1.101843e-01,5.445165e-01,-1.507289e-01,-2.659096e-01,2.596911e-01,-3.046813e-01,-1.917887e-01,-1.718476e-01,-2.253637e-02,-5.260161e-02,-2.974124e-01,-7.938717e-02,-2.596901e-01,-1.962994e-01,-2.520893e-01,-1.961564e-01,-2.839126e-01,-2.255377e-01,-2.302942e-01,-1.439529e-01,-1.003158e-01,-1.263447e-01,-3.189250e

In [84]:
X[date_attribs].describe()

,r_date,h_birthday
count,240471,240471
unique,535,1673
top,2015-02-22,2012-04-30
freq,565,785


In [90]:
date_pipeline = Pipeline([
    ('date_attribs_adder', CombinedDateAttributesAdder()),
    ('imputer', SimpleImputer(strategy='median'))
])

pd.DataFrame(date_pipeline.fit_transform(X[date_attribs])).describe()

,0
count,2.404710e+05
mean,1.226805e+17
std,4.147111e+16
min,6.359040e+16
25%,9.218880e+16
50%,1.077408e+17
75%,1.462752e+17
max,3.742848e+17


In [9]:
X.r_dirt_condition.value_counts(dropna=False)

NaN               120464
good               70944
slightly_heavy     25888
heavy              15379
bad                 7796
Name: r_dirt_condition, dtype: int64

In [10]:
X.r_turf_condition.value_counts(dropna=False)

NaN               117982
good               98109
slightly_heavy     17023
heavy               6211
bad                 1146
Name: r_turf_condition, dtype: int64

In [11]:
from sklearn.preprocessing import OneHotEncoder

X['r_dirt_condition'] = X['r_dirt_condition'].fillna('')
X['r_turf_condition'] = X['r_turf_condition'].fillna('')

cat_attribs = [
    'r_racetrack', 'r_course_type', 'r_weather', 'h_sex', 
    'r_impost_category', 'r_dirt_condition', 'r_turf_condition'
]

cat_encoder = OneHotEncoder(handle_unknown='ignore')
cat_encoder.fit_transform(X[cat_attribs]).toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [12]:
cat_encoder.categories_

[array(['chukyo', 'fuma', 'hakodate', 'hanshin', 'kyoto', 'nakayama',
        'niigata', 'ogura', 'sapporo', 'tokyo'], dtype=object),
 array(['dirt', 'obstacle', 'turf'], dtype=object),
 array(['cloudy', 'rainy', 'snowy', 'sunny'], dtype=object),
 array(['castrated', 'female', 'male'], dtype=object),
 array(['age_based', 'age_sex_based', 'decided_per_race', 'handicap'],
       dtype=object),
 array(['', 'bad', 'good', 'heavy', 'slightly_heavy'], dtype=object),
 array(['', 'bad', 'good', 'heavy', 'slightly_heavy'], dtype=object)]